In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score

C:\Users\harsh\AppData\Local\Temp\ipykernel_4764\3401383608.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load data from CSV files
train_features = pd.read_csv('Dataset/training_set_features.csv')
train_labels = pd.read_csv('Dataset/training_set_labels.csv')
test_features = pd.read_csv('Dataset/test_set_features.csv')
submission_format = pd.read_csv('Dataset/submission_format.csv')

In [3]:
# Display the first few rows of the data
print("Training features preview:")
print(train_features.head())

print("Training labels preview:")
print(train_labels.head())

Training features preview:
   respondent_id  xyz_concern  xyz_knowledge  behavioral_antiviral_meds  \
0              0          1.0            0.0                        0.0   
1              1          3.0            2.0                        0.0   
2              2          1.0            1.0                        0.0   
3              3          1.0            1.0                        0.0   
4              4          2.0            1.0                        0.0   

   behavioral_avoidance  behavioral_face_mask  behavioral_wash_hands  \
0                   0.0                   0.0                    0.0   
1                   1.0                   0.0                    1.0   
2                   1.0                   0.0                    0.0   
3                   1.0                   0.0                    1.0   
4                   1.0                   0.0                    1.0   

   behavioral_large_gatherings  behavioral_outside_home  \
0                          0.0

In [4]:
# Handle missing values by imputing with the most frequent value
imputer = SimpleImputer(strategy='most_frequent')
train_features_filled = imputer.fit_transform(train_features)
train_features_filled = pd.DataFrame(train_features_filled, columns=train_features.columns)

In [5]:
# Identify and encode categorical columns
categorical_cols = train_features_filled.select_dtypes(include=['object']).columns
label_encoders = {col: LabelEncoder().fit(train_features_filled[col].astype(str)) for col in categorical_cols}

for col, le in label_encoders.items():
    train_features_filled[col] = le.transform(train_features_filled[col].astype(str))


In [6]:
# Ensure there are no missing values after imputation
assert train_features_filled.isnull().sum().sum() == 0, "There are still missing values!"

In [7]:

# Define feature matrix X and target matrix y
X = train_features_filled
y = train_labels[['xyz_vaccine', 'seasonal_vaccine']]


In [8]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=7)

In [10]:
# Train the model with RandomForestClassifier wrapped in MultiOutputClassifier
rf = RandomForestClassifier(random_state=42)
multi_output_rf = MultiOutputClassifier(rf, n_jobs=-1)
multi_output_rf.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42),
                      n_jobs=-1)

In [11]:

# Predict probabilities on the validation set
y_val_pred = multi_output_rf.predict_proba(X_val)

In [12]:
# Extract probabilities for each target
y_val_pred_xyz = [pred[1] for pred in y_val_pred[0]]
y_val_pred_seasonal = [pred[1] for pred in y_val_pred[1]]


In [13]:
# Calculate ROC AUC scores
roc_auc_xyz = roc_auc_score(y_val['xyz_vaccine'], y_val_pred_xyz)
roc_auc_seasonal = roc_auc_score(y_val['seasonal_vaccine'], y_val_pred_seasonal)
mean_roc_auc = (roc_auc_xyz + roc_auc_seasonal) / 2

print(f"ROC AUC for xyz_vaccine: {roc_auc_xyz}")
print(f"ROC AUC for seasonal_vaccine: {roc_auc_seasonal}")
print(f"Mean ROC AUC: {mean_roc_auc}")

ROC AUC for xyz_vaccine: 0.8281476153459923
ROC AUC for seasonal_vaccine: 0.8466387453417549
Mean ROC AUC: 0.8373931803438737


In [ ]:
# Predict probabilities on the test set
y_test_pred = multi_output_rf.predict_proba(test_features_scaled)

y_test_pred_xyz = [pred[1] for pred in y_test_pred[0]]
y_test_pred_seasonal = [pred[1] for pred in y_test_pred[1]]

In [ ]:
# Create and save the submission file
submission = pd.DataFrame({
    'respondent_id': submission_format['respondent_id'],
    'xyz_vaccine': y_test_pred_xyz,
    'seasonal_vaccine': y_test_pred_seasonal
})

submission.to_csv('submission.csv', index=False)
print("Submission file created.")